In [1]:
import h5py

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
SHAPENET_PATH = '/home/ankbzpx/datasets/'
path_mesh = SHAPENET_PATH + 'ShapeNet/Mesh/03001627/'
path_model = SHAPENET_PATH + 'ShapeNet/ShapeNetCore.v1/03001627/'
path_voxel = SHAPENET_PATH + 'ShapeNet/ShapeNetVox32/03001627/'
path_depth = SHAPENET_PATH + 'ShapeNet/ShapeNetDepth/03001627/'

hidden_dim_discrete = 128

# data preparation
data_file = '/home/ankbzpx/datasets/ShapeNet/ShapeNetRenderingh5_v1/03001627/data_test.h5'
sample_size = 2048
batch_size = 4
split_ratio = 0.9
depth_size = 256
num_of_workers = 12

In [4]:
from mesh_to_sdf import sample_sdf_near_surface
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import transforms
import torch.nn as nn
import torchvision.transforms.functional as TF
from torch.autograd import Variable
from PIL import Image

import time
import pymesh
import trimesh

In [5]:
# reproducible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
 
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
    
    
#device = torch.device("cpu")
torch.manual_seed(0)
np.random.seed(0)

In [6]:
device

device(type='cuda')

In [7]:
class ChairDepthDataset(Dataset):
    
    def __init__(self, h5_file, transform=None):
        
        self.hf = h5py.File(h5_file, 'r')
        self.keys = list(self.hf.keys())
        
        self.transform = transform

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, idx):
        
        group = self.hf[self.keys[idx]]
        
        model_id = self.keys[idx]
        depth_img = Image.fromarray(np.array(group['depth_img']))
        azimuth = float(group['azimuth'][()])
        elevation = float(group['elevation'][()])
        distance = float(group['distance'][()])
        target_vox = torch.tensor(group['target_vox'], dtype=torch.float)
        
        if self.transform:
            depth_img = self.transform(depth_img)
        
        depth_img = nn.functional.interpolate(depth_img.unsqueeze(0), size=256, mode='nearest').squeeze(0)
        
        sample = {'model_id': model_id,
                  'depth_img': depth_img,
                  'azimuth': azimuth,
                  'elevation': elevation,
                  'distance': distance,
                  'target_vox': target_vox, 
                 }

        return sample

In [8]:
test_depth_dataset = ChairDepthDataset(data_file,
                         transforms.Compose([
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5, ), (0.5, ))
                                ]))

test_size = len(test_depth_dataset)

print('Test size: ', test_size)

test_depth_dataloader = DataLoader(test_depth_dataset, batch_size=batch_size, shuffle=False, num_workers=num_of_workers)

Test size:  31464


In [14]:
# Pre-trained Model path

MODEL_PATH = '/home/ankbzpx/Documents/'
discrete_encoder_path = MODEL_PATH + 'discrete continuous shape/discrete_encoder_model_256.pth'
discrete_decoder_path = MODEL_PATH + 'discrete continuous shape/discrete_decoder_model_256.pth'
unet_path = MODEL_PATH + 'discrete continuous shape/unet.pth'
continuous_model_path = MODEL_PATH + 'discrete continuous shape/continuous_model.pth'


from models import Discrete_encoder, Discrete_decoder, UNET, Continuous

####################
# Discrete Encoder #
####################

discrete_encoder = Discrete_encoder(256).to(device)
discrete_encoder.load_state_dict(torch.load(discrete_encoder_path))
discrete_encoder.eval()

for child in discrete_encoder.children():
    for param in child.parameters():
        param.requires_grad = False

####################
# Discrete Decoder #
####################
        
discrete_decoder = Discrete_decoder(256).to(device)
discrete_decoder.load_state_dict(torch.load(discrete_decoder_path))
discrete_decoder.eval()

for child in discrete_decoder.children():
    for param in child.parameters():
        param.requires_grad = False
        
########
# UNET #
########

# pre-trained model is loaded within the model
unet = UNET(unet_path).to(device)

unet.eval()

for child in unet.children():
    for param in child.parameters():
        param.requires_grad = False

##############
# Continuous #
##############

continuous = Continuous().to(device)
continuous.load_state_dict(torch.load(continuous_model_path))
continuous.eval()

for child in continuous.children():
    for param in child.parameters():
        param.requires_grad = False

## Generation

In [26]:
# plot vox or img
from vis_utils import plotFromVoxels, plotImg
# generate with continuous model
from render_utils import generate_mesh, get_normalize_mesh
# render mesh with pyrender
from render_utils import render
# mesh from ground truth vox
from render_utils import mesh_from_voxel
# transfrom sdf
from render_utils import get_transformed_indices, sdf2Voxel
# get cd, emd, iou from 2 pymesh
from render_utils import get_test_results

In [22]:
idx = 20000


data = test_depth_dataset[idx]

test_depth_img = data['depth_img'].to(device)
z = discrete_encoder(test_depth_img.unsqueeze(0))
out_vox = torch.sigmoid(discrete_decoder(z))

azimuth = data['azimuth']
elevation = data['elevation']

print('Azimuth: ', azimuth)
print('Elevation: ', elevation)

mesh = generate_mesh(continuous, unet, out_vox, z, device, grid_res = 64, azimuth = azimuth, elevation = elevation, isosurface = 0)

render(mesh)

Azimuth:  80.5938268218
Elevation:  28.2788330136


In [20]:
import os

start_time = time.time()

for data in test_depth_dataset:
    
    start_time = time.time()
    
    dir_name, _id = data['model_id'].split('_')
    
    if int(_id) == 0:
        
        # Create output folder
        outputFolder = path_mesh + dir_name
        
        if not os.path.exists(outputFolder):
            os.makedirs(outputFolder)
        
        test_depth_img = data['depth_img'].to(device)
        z = discrete_encoder(test_depth_img.unsqueeze(0))
        out_vox = torch.sigmoid(discrete_decoder(z))
        
        azimuth = data['azimuth']
        elevation = data['elevation']
        mesh_py = generate_mesh(continuous, unet, out_vox, z, device, grid_res = 64, azimuth = azimuth, elevation = elevation, isosurface = 0)
        
        obj_file =  outputFolder + '/' + _id + '.obj'
        print(obj_file)
        
        pymesh.save_mesh_raw(obj_file, mesh_py.vertices, mesh_py.faces)
        
        print("--- %s seconds ---" % (time.time() - start_time))

/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/103b75dfd146976563ed57e35c972b4b/0.obj
--- 2.7838082313537598 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/103d77d63f0d68a044e6721e9db29c1b/0.obj
--- 2.987065315246582 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/1049953406c81b237eaeab1f0c9120b7/0.obj
--- 2.7150707244873047 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/106a0dbaead5066519fb4103277a6b93/0.obj
--- 2.7076172828674316 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/112cee32461c31d1d84b8ba651dfb8ac/0.obj
--- 2.767735481262207 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/11506b96d41f7d3dd7c4a943f33e0384/0.obj
--- 2.735137462615967 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/117bd6da01905949a81116f5456ee312/0.obj
--- 2.7605319023132324 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/11a06e6f68b1d99c8687ff9b0b4e4ac/0.obj
--- 2.8904638290405273 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/0300162

/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/225ef5d1a73d0e24febad4f49b26ec52/0.obj
--- 2.8835949897766113 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/226704c72560008421ceb39dc3069834/0.obj
--- 2.866072416305542 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/22ada577361ed0374b3c42e318f3affc/0.obj
--- 2.87634539604187 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/22af872ac796ed26ff8d7c1096fae070/0.obj
--- 2.9063756465911865 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/22b8498e1ee46520737a00f007529fbf/0.obj
--- 2.935178518295288 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/2343e2c4fa69f33a2ff834514c92e8fd/0.obj
--- 2.9268765449523926 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/2403b6769a03c8a466ab323d8f805a57/0.obj
--- 2.922917366027832 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/247dd48e4b6c029e4b3c42e318f3affc/0.obj
--- 2.8824474811553955 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627

/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/2f282d5e4f140119e493511b69cc95d0/0.obj
--- 3.0696680545806885 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/2f6b0ddf12d1311795bea7c29e873d16/0.obj
--- 3.051978349685669 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/2fed64c67552aa689c1db271ad9472a7/0.obj
--- 3.0522735118865967 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/3010d6ca1b1e8ce3bf54ca9b2f215141/0.obj
--- 3.0489087104797363 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/30378faa6bf5b245fdef1c01cbd4ae0c/0.obj
--- 3.0771751403808594 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/30bbee7ab9026c5e4b3c42e318f3affc/0.obj
--- 2.9935994148254395 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/30beaf15d2d2beb1febad4f49b26ec52/0.obj
--- 2.9632091522216797 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/30fafef5c734f926781ba0fdb47276df/0.obj
--- 2.975368022918701 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001

/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/3b2710b509c3df0ea5e8cf3aa1686d66/0.obj
--- 3.058900833129883 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/3b3a9f4e3aa9f2f4d39a194653571dfc/0.obj
--- 2.9916396141052246 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/3b4292989394ba62f51f77a6d7299806/0.obj
--- 3.040031909942627 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/3b788994cd578990c35131da26f8061a/0.obj
--- 2.9626331329345703 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/3c27660aacbcf99886327adaa986dff/0.obj
--- 2.989987373352051 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/3c3b33293af7459f2677042d71d00b91/0.obj
--- 2.97706937789917 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/3c786ff99885e95c685d4893e4ba8951/0.obj
--- 2.9527714252471924 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/3c81fab5678a3872327289c00b6dc9ca/0.obj
--- 3.001779794692993 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/3

/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/4428b7dc4b6696812905b6e26038a78/0.obj
--- 3.520209789276123 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/445d6cef46b853ee713decb1a0563b12/0.obj
--- 3.33016300201416 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/4499729e53c858ae71a782a4379556c7/0.obj
--- 3.07810115814209 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/44a2a3952ea2315ff51f77a6d7299806/0.obj
--- 3.07712459564209 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/44ce5f9acd418c7dfebad4f49b26ec52/0.obj
--- 3.2335848808288574 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/44ddb3d46266bb0ffebad4f49b26ec52/0.obj
--- 3.3541698455810547 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/4519d91ba59f0267cc0484ea4f50062/0.obj
--- 3.7949140071868896 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/452115e132539be4daaaeef365d8f6e5/0.obj
--- 3.329155921936035 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/453a

...

/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/c7e590c0390e8d5debe67d9b32c3ddf8/0.obj
--- 2.94447922706604 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/c7fe45610d10cb108ad3a7d07aac2767/0.obj
--- 2.996623992919922 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/c80c0b8107583898c1956375af82917f/0.obj
--- 2.9937832355499268 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/c8265e04c94bcb5a1346e336f65f96f6/0.obj
--- 3.033259868621826 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/c826c65111c867ab45a1df43bcd9e471/0.obj
--- 2.982165813446045 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/c86cfe147872280463626070a93463cf/0.obj
--- 2.9823660850524902 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/c8938f54fecab41e77cd061c90fcdb44/0.obj
--- 2.9908666610717773 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/c8daa8e9496580667b9c6deef486a7d8/0.obj
--- 3.028285503387451 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/

/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/d239d38424429a9a4626612b5d655dc/0.obj
--- 2.9790632724761963 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/d2597d18fdc3594e1dc59d2adbe5297d/0.obj
--- 2.9459657669067383 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/d274fc14092387c1e17e1cb731e2fa4f/0.obj
--- 2.9874022006988525 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/d2815e678f173616e6cfc789522bfbab/0.obj
--- 3.052825689315796 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/d29445f24bbf1b1814c05b481f895c37/0.obj
--- 3.01070499420166 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/d2992fd5e6715bad3bbf93f83cbaf271/0.obj
--- 2.973792552947998 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/d29971cef754cc91cd8c5d1ba690a2c3/0.obj
--- 2.9715962409973145 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/d2af105ee87bc66dae981a300c94a911/0.obj
--- 2.9540293216705322 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627

/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/df2b7e697ab6ca0f155d75bbf62b80/0.obj
--- 2.963029384613037 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/df51cb83f0e55b81d85934de5898f0b/0.obj
--- 2.932988405227661 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/df609533cd186278398c7598b0d2e5d5/0.obj
--- 2.940568447113037 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/df7735e2bce09a511f98c0761af40e04/0.obj
--- 2.909372091293335 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/df8440d8678f3a91c8687ff9b0b4e4ac/0.obj
--- 2.9081246852874756 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/df8c98028d8ff2435dc3653f8341633a/0.obj
--- 2.9602975845336914 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/dfae4f9155877a1ef57b53e86a4ec824/0.obj
--- 2.8983571529388428 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/dfd92aab082e4915e3d9492680c47206/0.obj
--- 2.905141592025757 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/df

/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/e6f37dff25ec4ca4f815ebdb2df45512/0.obj
--- 3.0042285919189453 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/e779cf261cf04a77acd8c40fddcf9ca/0.obj
--- 3.0390288829803467 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/e803b31e2185d0405784b22e1081a3e1/0.obj
--- 3.0125322341918945 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/e807c535765fbbbab40f0ac0fb9a650d/0.obj
--- 2.984006404876709 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/e8126f9e2d106620d2f33aaf794b5932/0.obj
--- 2.9932734966278076 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/e84b5bbe91e0636cb21bc3cf138f79e/0.obj
--- 3.016268730163574 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/e8788e6f0292f503f4b6538438a0b930/0.obj
--- 2.9892404079437256 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/e89689500a1e3b95fa935eb36a2e89fe/0.obj
--- 2.9624743461608887 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/0300162

/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/f2dd3bc7d6c85a8f2813aa30be7ebdbc/0.obj
--- 3.0300893783569336 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/f2e2993abf4c952b2e69a7e134f91051/0.obj
--- 3.0361971855163574 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/f33b6f791e9d64387d01b77e04a0bc7b/0.obj
--- 3.0335817337036133 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/f3573756e64259f2b29d280b4e59c527/0.obj
--- 3.059330940246582 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/f36e83dff72cba53ba9ae5e8f97b3d4/0.obj
--- 3.0145034790039062 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/f3f31db17715ee0c327289c00b6dc9ca/0.obj
--- 3.0324535369873047 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/f3fa7bd00b76f6a87a8a6b9421844d96/0.obj
--- 3.016655921936035 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627/f444df40dadd83e22cedd65b2f6d1fdf/0.obj
--- 3.01296329498291 seconds ---
/home/ankbzpx/datasets/ShapeNet/Mesh/03001627

## Evaluation

In [ ]:
import time
from glob import glob

count = 32768
dim = 64

mesh_dir_list = sorted(glob(path_mesh + '*/'))

#cf_model_all = {}
#emd_model_all = {}
#iou_model_all = {}

cf_model = []
emd_model = []
iou_model = []

for idx in range(len(mesh_dir_list)):
    
    start_time = time.time()
    print(idx)
    
    dir_name = mesh_dir_list[idx].split('/')[-2]
    
    gt_mesh_path = path_model + dir_name + '/model.obj'
    gt_mesh_py = pymesh.load_mesh(gt_mesh_path)
    
    transformed_vertices = get_transformed_indices(gt_mesh_py.vertices, 0, 0)
    gt_mesh_py = pymesh.form_mesh(transformed_vertices, gt_mesh_py.faces)
    gt_mesh_py = get_normalize_mesh(gt_mesh_py)
    
    view_idx = 0
    
    #for view_idx in range(24):
    
    test_mesh = mesh_dir_list[idx] + str(view_idx) + '.obj'
    test_mesh_py = pymesh.load_mesh(test_mesh)
    
    chamfer_dist, emd, iou = get_test_results(gt_mesh_py, test_mesh_py)
    
    cf_model.append(chamfer_dist)
    emd_model.append(emd)
    iou_model.append(iou)
    
    
    #cf_model_all[dir_name] = np.average(cf_model)
    #emd_model_all[dir_name] = np.average(emd_model)
    #iou_model_all[dir_name] = np.average(iou_model)
    
    print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
import time
from glob import glob

count = 32768
dim = 64

with open(SHAPENET_PATH + 'ShapeNet/fileList/03001627_test.lst', 'r') as f:
    index_dir = f.read().splitlines()

cf_model_voxel = []
emd_model_voxel = []
iou_model_voxel = []

for idx in range(len(index_dir)):
    
    start_time = time.time()
    print(idx)
    
    voxel_dir = path_voxel + index_dir[idx] + '/'
    model_dir = path_model + index_dir[idx] + '/'
    mesh_dir = path_mesh + index_dir[idx] + '/'
    
    #################################
    
    gt_mesh_path = model_dir + 'model.obj'
    gt_mesh_py = pymesh.load_mesh(gt_mesh_path)
    transformed_vertices = get_transformed_indices(gt_mesh_py.vertices, 0, 0)
    gt_mesh_py = pymesh.form_mesh(transformed_vertices, gt_mesh_py.faces)
    gt_mesh_py = get_normalize_mesh(gt_mesh_py)
    
    gt_samples_batch, gt_samples, v1 = mesh_test(gt_mesh_py)
    
    
    ########################################
    
    voxel_mesh = mesh_from_voxel(voxel_dir)
    voxel_mesh_py = pymesh.form_mesh(voxel_mesh.vertices, voxel_mesh.faces)
    
    voxel_samples_batch, voxel_samples, v3 = mesh_test(gt_mesh_py)
    
    
    ########################################
    
    get_test_results([gt_samples_batch, voxel_samples_batch], [gt_samples, voxel_samples], [v1, v3])
    
    cf_model_voxel.append(chamfer_dist)
    emd_model_voxel.append(emd)
    iou_model_voxel.append(iou)
    
    print("--- %s seconds ---" % (time.time() - start_time))

In [33]:
print('Continuous vs. mesh')
print('Chamfer Distacne')
print('Mean: ', np.mean(cf_model))
print('Std: ', np.std(cf_model))
print()
print('Earth Movers Distance')
print('Mean: ', np.mean(emd_model))
print('Std: ', np.std(emd_model))
print()
print('Intersection over Union')
print('Mean: ', np.mean(iou_model))
print('Std: ', np.std(iou_model))

Continuous vs. mesh
Chamfer Distacne
Mean:  0.016240275
Std:  0.010957088

Earth Movers Distance
Mean:  0.031092420137995635
Std:  0.01829026307098186

Intersection over Union
Mean:  0.49419414573553655
Std:  0.16526782001919169


In [31]:
print('Voxel vs. mesh')
print('Chamfer Distacne')
print('Mean: ', np.mean(cf_model_voxel))
print('Std: ', np.std(cf_model_voxel))
print()
print('Earth Movers Distance')
print('Mean: ', np.mean(emd_model_voxel))
print('Std: ', np.std(emd_model_voxel))
print()
print('Intersection over Union')
print('Mean: ', np.mean(iou_model_voxel))
print('Std: ', np.std(iou_model_voxel))

Voxel vs. mesh
Chamfer Distacne
Mean:  0.016645646
Std:  0.0067713535

Earth Movers Distance
Mean:  0.04299999420118475
Std:  0.019341245953544087

Intersection over Union
Mean:  0.48158108402541405
Std:  0.1449197042451776


In [28]:
cf = list(cf_model_all.values())
emd = list(emd_model_all.values())
iou = list(iou_model_all.values())

In [32]:
print('Chamfer Distacne')
print('Mean: ', np.mean(cf))
print('Std: ', np.std(cf))
print()
print('Earth Movers Distance')
print('Mean: ', np.mean(emd))
print('Std: ', np.std(emd))
print()
print('Intersection over Union')
print('Mean: ', np.mean(iou))
print('Std: ', np.std(iou))

Chamfer Distacne
Mean:  0.01863253
Std:  0.010610847

Earth Movers Distance
Mean:  0.03635404172065211
Std:  0.01896437826828838

Intersection over Union
Mean:  0.47859730705525155
Std:  0.16438246008902013
